In [1]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
import numpy as np
posting_list = [
        ['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
        ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
        ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
        ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
        ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
        ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid'],
        ['i', 'want', 'to', 'hear', 'stories', 'about', 'your', 'dogs'],
        ['shut', 'up', 'you', 'idiot']
    ]
class_vec = [0, 1, 0, 1, 0, 1, 0, 1]

test_set = [
    ['love', 'my', 'dalmation'],
    ['stupid', 'garbage'],
    ['stop', 'posting', 'about', 'your', 'idiot', 'dog']
]

In [2]:
def createVocabList(data_set):
    vocab_set = set([])
    for document in data_set:
        vocab_set = vocab_set | set(document)
    return list(vocab_set)

In [3]:
def setOfWords2Vec(vocab_list, input_set):
    return_vec = [0] * len(vocab_list)
    for word in input_set:
        assert word in vocab_list, "the word: %s is not in my Vocabulary!" % (word,)
        return_vec[vocab_list.index(word)] = 1
    return return_vec

In [4]:
vocab_list = createVocabList(posting_list)
X = np.array(list(map(lambda post: setOfWords2Vec(vocab_list, post), posting_list)))
Y = np.array(class_vec)
# clf = MultinomialNB()
clf = BernoulliNB()
clf.fit(X, Y)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [5]:
test = np.array(list(map(lambda post: setOfWords2Vec(vocab_list, post), test_set)))
clf.predict(test)

array([0, 1, 1])